In [1]:
# BW 2019
import pandas as pd
import numpy as np
import funciones as fn
import re
import codecs
import string
'''
import nltk
from nltk.corpus import stopwords
spanish_stopwords=stopwords.words('spanish')
spanish_stopwords.extend(['ser', 'haber', 'q'])
spanish_stopwords.remove('no')
spanish_stopwords.remove('ni')
'''

"\nimport nltk\nfrom nltk.corpus import stopwords\nspanish_stopwords=stopwords.words('spanish')\nspanish_stopwords.extend(['ser', 'haber', 'q'])\nspanish_stopwords.remove('no')\nspanish_stopwords.remove('ni')\n"

In [ ]:
df = pd.read_excel(r'E:\Base bonita\BW 2019 - Mayo 1 a Diciembre 31\Semana 1.xlsx')
df = fn.limpieza_base(df)
df = clean_cols(df)
df

In [2]:
def clean_cols(df):
    del_cols=["Query_Name", "Url", "Page_Type", "Language", "Country_Code", "Continent", "Continent_Code", 
          "Country", "Added", "Assignment", "Avatar", "Category_Details", 
          "Checked", "Display_URLs", "Expanded_URLs", "Facebook_Author_ID", "Facebook_Comments",
          "Facebook_Likes", "Facebook_Role", "Facebook_Shares", "Facebook_Subtype",
          "Full_Text", "Full_Name", "Instagram_Comments", "Instagram_Followers", "Instagram_Following",
          "Instagram_Likes", "Instagram_Interactions_Count", "Instagram_Posts",
          "Last_Assignment_Date", "Latitude", "Location_Name", "Longitude", "Media_Filter",
          "Media_URLs", "Original_Url", "Priority", "Short_URLs", "Resource_Id", "Starred",
          "Status", "Subtype", "Thread_Created_Date", "Thread_Id",
          "Thread_URL", "Twitter_Author_ID", "Twitter_Channel_Role", "Twitter_Reply_to", "Twitter_Retweet_of", 
          "Updated", "Blog_Name", "Copyright", "Item_Review", "Page_Type_Name", "Linkedin_Comments", 
          "Linkedin_Engagement", "Linkedin_Impressions", "Linkedin_Likes", "Linkedin_Shares", "Linkedin_Sponsored", 
          "Linkedin_Video_Views", "Parent_Blog_Name", "Parent_Post_Id", "Pub_Type", "Publisher_Sub_Type", "Rating", 
          "Reddit_Score", "Region", "Region_Code", "Root_Blog_Name", "Root_Post_Id", "Weblog_Title", "Title", "Tags"]
    pruebas = codecs.open('Columnas_a_eliminar.txt','r',encoding='utf-8')
    more_cols = list()
    for line in pruebas.readlines():
        more_cols.append(line.strip())
    del_cols = del_cols + more_cols
    df = df.drop(del_cols, axis=1)
    return df

In [3]:
#Tomemos solo la base marcada
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;-]')
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = deEmojify(text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = ' '.join(word for word in text.split() if word not in spanish_stopwords) # delete stopwors from text
    return text
def text_cleaner(df):
    df['Clean_Text']=df['Snippet'].replace(r"http\S+", '', regex=True)
    df['Clean_Text']=df['Clean_Text'].replace('#\\w+', '', regex=True)
    df['Clean_Text']=df['Clean_Text'].replace(r'([^0-9a-zA-Z])\1+', r'\1', regex=True)
    df['Clean_Text']=df['Clean_Text'].replace(r'^[^0-9a-zA-Z@#]+', '', regex=True)
    df['Clean_Text']=df['Clean_Text'].replace(r'()\1+', r'\1\1', regex=True)
    df['Clean_Text']=df['Clean_Text'].replace(r'(.)\1+', r'\1\1', regex=True)
    df['Clean_Text']=df['Clean_Text'].replace('"', '', regex=True)
    df['Clean_Text']=df['Clean_Text'].replace(r'@\w+', '', regex=True)
    df['Clean_Text'] = df['Clean_Text'].map(lambda x: re.sub('[,\.!?]', '', x))
    df['Clean_Text'] = df['Clean_Text'].map(lambda x: x.lower())
    df=df.reset_index(drop=True)
    df = df.astype({"Clean_Text": str})
    df['Clean_Text']=df['Clean_Text'].str.strip()
    df['Clean_Text'] = df['Clean_Text'].replace("", np.nan)
    #df['Clean_Text_No_Stopwords'] = df['Clean_Text'].apply(clean_text)
    return df

In [4]:
path = r'E:\Base bonita\BW 2019 - Mayo 1 a Diciembre 31'
def unir(bases):
    i = 1
    final = pd.DataFrame()
    while i <= bases:
        j = str(i)
        est = path + "\Semana " + j + ".xlsx"
        print("---------- Abriendo base: " + "Semana " + j + " ------------"  )
        df = pd.read_excel(est)
        df = fn.limpieza_base(df)
        df = clean_cols(df)
        print("Completada la limpieza")
        final = final.append(df)
        print("Pegada a base final")
        print(final.shape[0])
        i += 1
    final = text_cleaner(final)
    #final = final.set_index(final["ID"]).drop(["ID"], axis=1).fillna(0)
    return final

In [5]:
final = unir(33)
final

C:\Users\JOSE\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


---------- Abriendo base: Semana 1 ------------
Completada la limpieza
Pegada a base final
28232
---------- Abriendo base: Semana 2 ------------
Completada la limpieza
Pegada a base final
46352
---------- Abriendo base: Semana 3 ------------
Completada la limpieza
Pegada a base final
62888
---------- Abriendo base: Semana 4 ------------
Completada la limpieza
Pegada a base final
76121
---------- Abriendo base: Semana 5 ------------
Completada la limpieza
Pegada a base final
90500
---------- Abriendo base: Semana 6 ------------
Completada la limpieza
Pegada a base final
112824
---------- Abriendo base: Semana 7 ------------
Completada la limpieza
Pegada a base final
130605
---------- Abriendo base: Semana 8 ------------
Completada la limpieza
Pegada a base final
149578
---------- Abriendo base: Semana 9 ------------
Completada la limpieza
Pegada a base final
169842
---------- Abriendo base: Semana 10 ------------
Completada la limpieza
Pegada a base final
182100
---------- Abriendo base

,Date,Snippet,Domain,Sentiment,City_Code,Account_Type,Author,City,Gender,Hashtags,...,Categorías_prueba_-_denuncia_xenofobia_col,Departamentos_-_Atlantico_(Barranqulla),Departamentos_-_Cundinamarca,Departamentos_-_Meta_(Villavicencio),Departamentos_-_Valle_del_Cauca_(Cali),Engagement_Type,xenofobia_e_Integración_-_Integración,xenofobia_e_Integración_-_Xenofobia,xenofobia_e_Integración_-_not_xenofobia,Clean_Text
0,2019-05-07 23:59:31.0,@Aracno_Red Creo que todo se volvió un relajo ...,twitter.com,positive,COL.Bogota.Bogota,NaN,Esenderista,Bogota,male,NaN,...,NaN,NaN,NaN,NaN,NaN,REPLY,NaN,NaN,X,creo que todo se volvió un relajo desde que pa...
1,2019-05-07 23:59:24.0,RT @petrogustavo Ordoñez comienza su campaña c...,twitter.com,negative,COL.Bogota.Bogota,individual,seafight3,Bogota,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,RETWEET,NaN,NaN,NaN,rt ordoñez comienza su campaña como todas las...
2,2019-05-07 23:59:03.0,RT @petrogustavo Ordoñez comienza su campaña c...,twitter.com,negative,COL.Bogota.Bogota,NaN,Antonio30said,Bogota,male,NaN,...,NaN,NaN,NaN,NaN,NaN,RETWEET,NaN,NaN,NaN,rt ordoñez comienza su campaña como todas las...
3,2019-05-07 23:58:00.0,RT @FelicianoValen En Washington la diplomacia...,twitter.com,neutral,COL.Bogota.Bogota,NaN,DanielRinconY,Bogota,male,NaN,...,NaN,NaN,NaN,NaN,NaN,RETWEET,NaN,NaN,NaN,rt en washington la diplomacia colombiana se ...
4,2019-05-07 23:57:41.0,RT @Presidencia_VE El Presidente (E) @jguaido ...,twitter.com,neutral,COL.Bogota.Bogota,NaN,jequintana,Bogota,male,NaN,...,NaN,NaN,NaN,NaN,NaN,RETWEET,NaN,NaN,NaN,rt el presidente (e) ha iniciado el proceso ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570411,2019-12-23 00:03:51.0,RT @_La_Neira Anoche robaron el bus donde iba ...,twitter.com,negative,COL.Bogota.Bogota,individual,Tatawitti,Bogota,male,NaN,...,NaN,NaN,NaN,NaN,NaN,RETWEET,NaN,NaN,NaN,rt anoche robaron el bus donde iba una de mis...
570412,2019-12-23 00:02:09.0,RT @robinso38758682 @ThinKarawak @marco_caldon...,twitter.com,negative,COL.Bogota.Bogota,individual,NpijdMuoz,Bogota,male,NaN,...,NaN,NaN,NaN,NaN,NaN,RETWEET,NaN,NaN,NaN,rt bueno hermanos venezolanos hay que apo...
570413,2019-12-23 00:01:47.0,"RT @HERCULESBACKUS ¡Mr. Mark Esper, es el mome...",twitter.com,neutral,COL.Valle del Cauca.Cali,individual,fernandomenshuk,Cali,male,NaN,...,NaN,NaN,NaN,NaN,NaN,RETWEET,NaN,NaN,X,rt ¡mr mark esper es el momento de pasar de l...
570414,2019-12-23 00:01:02.0,@gbobadi @SANGREXENEIZE Y el acento (incluso l...,twitter.com,neutral,COL.Bogota.Bogota,individual,ArevaloJuanP,Bogota,male,NaN,...,NaN,NaN,NaN,NaN,NaN,REPLY,NaN,NaN,X,y el acento (incluso la voz chillona de chicho...


In [6]:
final.to_excel("Base_BW_2019.xlsx")